In [ ]:
import os
import numpy as np
import cv2
import imutils
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# Test mode is set to false to avoid unwated testing outputs from the cells
test = False

The Brain tumor dataset is taken from `./brain_tumor_dataset/yes` and `brain_tumor_dataset/no` directories. 
> **Dataset Source:** https://www.kaggle.com/navoneel/brain-mri-images-for-brain-tumor-detection 
> 
> **Dataset Description**: The dataset contains 2 folders (yes/no) and each folder contains subfolders with several images. There are 253 images with brain tumor and 98 images without brain tumor. The images are in .jpg format. The image sizes are not consistent.

In [ ]:
YES_DIR_PATH = 'brain_tumor_dataset/yes/'
NO_DIR_PATH = 'brain_tumor_dataset/no/'
yes_imgs_name = os.listdir(YES_DIR_PATH)
no_imgs_name = os.listdir(NO_DIR_PATH)


# 1. Pre-processing the images

  1. Crop the brain out of the images.
  2. Resize the images to a standard size of 32x32 for easier processing.
  3. Normalize pixels of the images

### 1.1 Cropping the brain out of the images
Create a function to crop the brain out an given image

![MRI Cropping Procedure](./MRIcropping.png)
<!-- For kaggle drag and drop MRIcropping.png into the notebook markdown -->

  0. Resize the image to 256x256 pixels
  1. Convert the image to grayscale
  2. Apply a Gaussian blur to the image
  3. Threshold the image and perform a series of erosions and dilations to remove any small regions of noise
  4. Find the contours of the brain
  5. Find the largest contour
  6. Crop the image to the bounding box of the largest contour

In [ ]:
def crop_image(img):
    # Resize the image to 256x256 pixels
    resized_img = cv2.resize(
        img,
        dsize=(256, 256),
        interpolation=cv2.INTER_CUBIC
    )

    # Convert the image to grayscale
    gray = cv2.cvtColor(resized_img, cv2.COLOR_RGB2GRAY)

    # Apply a Gaussian blur to the image
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image by Binary Thresholding
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]

    # perform a series of erosions & dilations to remove any small regions of noise
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)

    # find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    # crop
    ADD_PIXELS = 0
    cropped_img = resized_img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS,
                              extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
    return cropped_img


In [ ]:
# Use crop_image function to crop the images
yes_imgs_cropped = [crop_image(cv2.imread(YES_DIR_PATH + img_file)) for img_file in yes_imgs_name]
no_imgs_cropped = [crop_image(cv2.imread(NO_DIR_PATH + img_file)) for img_file in no_imgs_name]

### 1.2 Resize the cropped images

The cropped images are resized to 32x32 for easier processing.

The `resized_imgs` list contains the resized images of the `yes_imgs_cropped` and `no_imgs_cropped` lists.


In [ ]:
orig_imgs = yes_imgs_cropped + no_imgs_cropped
resized_imgs = [cv2.resize(img, dsize=(32, 32), interpolation=cv2.INTER_CUBIC) for img in orig_imgs]

In [ ]:
X = np.squeeze(resized_imgs)

In [ ]:
if (test):
    print(type(X))
    print(X.shape)
    print(X)
    print(resized_imgs)


### 1.3 Normalize pixels of the images
Normalize each pixel of the images present in the numpy array `X` and save it as `X`.


In [ ]:
# normalize data
X = X.astype('float32')
X /= 255

if (test):
    print(X)


# 2. Creating and Training the CNN model
  1. Create the labels for the images
  2. Prepare the training and validation/testing data
  3. Create the CNN model and visualize the model summary
  4. Compile the model
  5. Train the model on the training data
  6. Evaluate the model

### 2.1. Create the labels for the images
The `img_labels` list stores the labels for each image. The labels are 0 for "no" and 1 for "yes".


In [ ]:
labels_yes = np.full(len(yes_imgs_name), 1)
labels_no = np.full(len(no_imgs_name), 0)

img_labels = np.concatenate([labels_yes, labels_no])

In [ ]:
if (test):
    print(img_labels.size, img_labels)


### 2.2 Prepare the training and validation/testing data

Prepare `x_train` and `x_valid` from the `X` image data list.
Prepare `y_train` and `y_valid` from the `img_labels` list.

As stated above, the dataset is divided into 2 directories: "yes" & "no". There are 253 images (155 images for "yes" and 98 images for "no"). We were storing the labels for each image in the `img_labels` list. The labels are 0 for "no" and 1 for "yes".

So we are separately storing the images of "yes" and "no" in the `yes_imgs` and `no_imgs` lists respectively and thier labels in the `yes_labels` and `no_labels` lists respectively.

We will divide those images into training and validation sets. We will use 80% of the images for training and 20% of the images for validation. So we will use 202 images for training and 51 images for validation. Out of 202 images, 124 images are of "yes" and 78 images are of "no". Similarly, out of 51 images, 31 images are of "yes" and 20 images are of "no".


In [ ]:
# Splitting the dataset into the Training set (i.e `x_train`) and Testing set/Validation set (i.e `x_valid`)
# We will also keep the original images of the validation set in `x_orig_valid` for visualization purposes
yes_imgs = X[:155]
no_imgs = X[155:]
yes_orig_imgs = orig_imgs[:155]
no_orig_imgs = orig_imgs[155:]

x_yes_train = yes_imgs[:124]
x_yes_valid = yes_imgs[124:]
x_yes_orig_valid = yes_orig_imgs[124:]

x_no_train = no_imgs[:78]
x_no_valid = no_imgs[78:]
x_no_orig_valid = no_orig_imgs[78:]

x_train = np.concatenate([x_yes_train, x_no_train]) # 80% of the images
x_valid = np.concatenate([x_yes_valid, x_no_valid]) # 20% of the images
x_orig_valid = np.concatenate([x_yes_orig_valid, x_no_orig_valid])

# Splitting the dataset labels for the Training set (i.e `y_train`) and Testing set/Validation set (i.e `y_valid`)
yes_labels = img_labels[:155]
no_labels = img_labels[155:]

y_yes_train = yes_labels[:124]
y_yes_valid = yes_labels[124:]

y_no_train = no_labels[:78]
y_no_valid = no_labels[78:]

y_train = np.concatenate([y_yes_train, y_no_train])
y_valid = np.concatenate([y_yes_valid, y_no_valid])


### 2.3.1 Create the CNN model

Prepare the CNN Sequential model with 3 convolutional layers and 2 dense layers. Use the `relu` activation function for the convolutional layers and `sigmoid` activation function for the last dense layer.

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=9,
          padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.45))

model.add(tf.keras.layers.Conv2D(
    filters=16, kernel_size=9, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Conv2D(
    filters=36, kernel_size=9, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.15))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# model.summary() # Uncomment to see the model summary. Model summary is already been shown the cell below in SVG representation


### 2.3.2 Visualize the model summary
Visualize the SVG representation of the model.

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))


### 2.4 Compile the model
Use the `binary_crossentropy` loss function and `adam` optimizer to compile model.

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['acc'])


### 2.5 Train the model on the training data
Train the model for 200 epochs with a batch size of 128. Use the `validation_data` parameter to pass the validation data.

In [ ]:
model.fit(x_train,
          y_train,
          batch_size=128,
          epochs=200,
          validation_data=(x_valid, y_valid),)

# Saving the trained model
model.save('brain_tumor_detection_model.h5')

### 2.6 Evaluate the model
Evaluate the model on the validation data and print the test accuracy.

In [ ]:
score = model.evaluate(x_valid, y_valid, verbose="0")

print('Test accuracy:', score[1])

# 3. Predicting the output
  1. Predict the output for the validation data
  2. Visualize the predicted output

Plot a random sample of 15 test images, their predicted labels and ground truth with their accuracy scores
Also, plot the original MRIs of those randomly selected images for visualization purposes

In [ ]:
labels = ["No", "Yes"]
y_hat = model.predict(x_valid)

no_of_prediction_samples = 15
random_indices = np.random.choice(
    x_valid.shape[0], size=no_of_prediction_samples, replace=False)
# Plot a random sample of 15 test images, with their predicted labels and ground truth
figure = plt.figure(figsize=(20, 13))
sub_title = "Random samples of 15 test images, with their predicted labels and ground truth"
figure.suptitle(sub_title, fontsize=20)
for i in range(no_of_prediction_samples):
    rand_index = random_indices[i]

    # Display each image
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_valid[rand_index]))

    # Set the title for each image
    prediction_val = y_hat[rand_index][0]
    predict_index = 0 if (prediction_val < 0.5) else 1
    true_index = y_valid[rand_index]
    prediction = labels[predict_index]
    truth = labels[true_index]
    title_color = "blue" if predict_index == true_index else "red"
    ax_title = "Prediction: {} ({:.2f})\nGround Truth: {}".format(
        prediction, prediction_val, truth)
    ax.set_title(ax_title, color=title_color)
plt.show()

# Show the Original MRIs of the randomly selected images
figure = plt.figure(figsize=(20, 8))
figure.suptitle("Original MRIs of those randomly selected images", fontsize=20)
for i in range(no_of_prediction_samples):
    # get the original image
    img = x_orig_valid[random_indices[i]]
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(img))
plt.show()
